In [1]:
import A_GIS
import pathlib

coder_system = A_GIS.File.read(file='/Users/ww5/test_2code/coder_system.md')
coder = A_GIS.Ai.Chatbot.init(provider='groq',model='llama2',system=coder_system)

reviewer_system = A_GIS.File.read(file='/Users/ww5/test_2code/reviewer_system.md')
reviewer = A_GIS.Ai.Chatbot.init(provider='groq',model='mixtral',system=reviewer_system)

coder_directive = '''You are requested to write: an A_GIS functional unit that scans a string with python code and converts all relative imports to absolute,
including the calls to those functions, returning the corrected code.

Example 1:

It would convert:

def a():
    from xyz import fun
    return fun()

To this:

def a():
    import xyz.fun
    return xyz.fun()

'''

reviewer_directive = '''You are requested to review: an A_GIS functional unit that scans a string with python code and converts all relative imports to absolute,
including the calls to those functions, returning the corrected code.'''

reviewer_content = ''
keep_state = False
responses=[]
for it in range(3):
    message_to_coder = ''
    if reviewer_content != '':
        message_to_coder += '\nYour last code was:\n' + coder_content + '\nThe code review is:\n' + reviewer_content + '\n---END OF PREVIOUS CONTEXT---\n'
    message_to_coder += coder_directive
    coder_infile = pathlib.Path(f'/Users/ww5/test_2code/coder_infile{it}.md')
    A_GIS.File.touch(path=coder_infile)
    A_GIS.File.write(content=message_to_coder,file=coder_infile)

    response = coder.chat(message_to_coder,keep_state=keep_state)
    responses += response
    coder_content = response['message']['content']
    coder_outfile = pathlib.Path(f'/Users/ww5/test_2code/coder_outfile{it}.md')
    A_GIS.File.write(content=coder_content,file=coder_outfile)

    reviewer_infile = pathlib.Path(f'/Users/ww5/test_2code/reviewer_infile{it}.md')
    message_to_reviewer = reviewer_directive + '\nThe programmer has submitted the following code:\n'+coder_content
    A_GIS.File.touch(path=reviewer_infile)
    A_GIS.File.write(content=message_to_reviewer,file=reviewer_infile)

    response = reviewer.chat(message_to_reviewer,keep_state=keep_state)
    responses += response
    reviewer_content = response['message']['content']
    reviewer_outfile = pathlib.Path(f'/Users/ww5/test_2code/reviewer_outfile{it}.md')
    A_GIS.File.write(content=reviewer_content,file=reviewer_outfile)


NotFoundError: Error code: 404 - {'error': {'message': 'The model `llama2-70b-4096` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'code': 'model_not_found'}}

In [4]:
import A_GIS

In [14]:
directive="""
You are a world-class test creator for Python programs in the 
A_GIS code system. A_GIS is a collection of Python adhering to 
functional programming tenets with special considerations for
reproducibility by supporting transformations of source using
`import A_GIS.*` to special hashed function names that ensure
backwards compatibility forever. For this reason, A_GIS ALWAYS
uses absolute imports, even in tests. For all modules.
"""

system=f"""
{directive} 

Here is an example.

### Instruction:

name:
    A_GIS.File.hash

code:

    def hash(*, file: type["pathlib.Path"]):
        import hashlib

        sha256_hash = hashlib.sha256()
        with open(file, "rb") as f:
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()

### Response:

tests:

    import pytest
    import A_GIS.File.hash
    import pathlib

    def test_determinism(tmp_path: pathlib.Path):
        test_content = b"Hello, World!"
        test_file = tmp_path / "test_file.txt"
        test_file.write_bytes(test_content)

        # Compute the hash twice for the same content
        first_hash = A_GIS.File.hash(file=test_file)
        second_hash = A_GIS.File.hash(file=test_file)

        # Verify that both hashes are the same
        assert (
            first_hash == second_hash
        ), "The hash function should produce the same output for the same input"

    def bit_difference(hash1: str, hash2: str) -> int:
        # Convert hex hashes to binary representation, stripping the '0b' prefix
        bin1 = bin(int(hash1, 16))[2:].zfill(256)
        bin2 = bin(int(hash2, 16))[2:].zfill(256)

        # Count the differing bits
        return sum(b1 != b2 for b1, b2 in zip(bin1, bin2))

    def test_avalanche_effect(tmp_path: pathlib.Path):
        # Create two temporary files with content that differs by a small amount
        original_content = b"Hello, World!"
        modified_content = b"hello, World!"  # Small change: 'H' -> 'h'

        original_file = tmp_path / "original.txt"
        modified_file = tmp_path / "modified.txt"

        original_file.write_bytes(original_content)
        modified_file.write_bytes(modified_content)

        # Compute hashes
        original_hash = A_GIS.File.hash(file=original_file)
        modified_hash = A_GIS.File.hash(file=modified_file)

        # Calculate bit difference
        difference = bit_difference(original_hash, modified_hash)

        # Assuming SHA-256 hash function, which produces 256-bit hash
        # We expect a significant difference, for simplicity, let's say at least 25% of the bits should differ
        expected_min_difference = 256 * 0.25  # 25% of 256 bits

        assert (
            difference >= expected_min_difference
        ), "A small change in input should significantly change the output hash."

    def test_collision_resistance(tmp_path: pathlib.Path):
        first_content = b"Hello, World!"
        second_content = b"Goodbye, World!"

        first_file = tmp_path / "first_file.txt"
        second_file = tmp_path / "second_file.txt"

        first_file.write_bytes(first_content)
        second_file.write_bytes(second_content)

        # Compute the hashes
        first_hash = A_GIS.File.hash(file=first_file)
        second_hash = A_GIS.File.hash(file=second_file)

        # Verify that the hashes are different
        assert (
            first_hash != second_hash
        ), "Different inputs should not produce the same hash"

"""

In [15]:
name='A_GIS.Code.distill'
code=A_GIS.File.read(file='../source/A_GIS/Code/distill/__init__.py')
#code=A_GIS.Code.distill(code=code)
print(A_GIS.Code.highlight(code=code,force=True))

def distill(*, code: str) -> str:
    """Distill the given Python code by removing docstrings, converting
    multiline string literals, and removing comments.

    This function parses the provided Python code into an abstract syntax tree (AST),
    and then walks through the AST. It identifies and blanks out all docstrings and
    multiline string literals. Additionally, due to the nature of AST parsing in Python,
    comments (which start with '#') are not included in the AST and hence are not present
    in the unparsed code, effectively removing them from the output.

    It's important to note that this removal of comments is a byproduct of how the Python
    parser and the AST handle comments, rather than an explicit action by this function.

    Args:
        code (str): A string representing the Python code to be distilled.

    Returns:
        str: A string representing the purified Python code, with docstrings, multiline string
             literals, and comments removed.



In [39]:
coder = A_GIS.Ai.Chatbot.init(system=directive,provider='groq',model='mixtral')

indented_code = A_GIS.Text.add_indent(text=code)
message = f"Write a pytest for the function name={name} with the following code:\n{indented_code}\n"+"\n"+"""
Assume you are writing a fully valid Python test file that is to be run with `pytest tests.py`, 
so put all comments behind # and do not include any markdown formatting!

Use the fully qualified import, `import A_GIS.Code.distill`.

Use triple single quotes for example code.

Create a test function for each indepedent thing to test.
"""
print(message)
result = coder.chat(message=message)
content = result["message"]["content"]
content = A_GIS.Code.extract_python(text=content)

Write a pytest for the function name=A_GIS.Code.distill with the following code:
    def distill(*, code: str) -> str:
        """Distill the given Python code by removing docstrings, converting
        multiline string literals, and removing comments.
    
        This function parses the provided Python code into an abstract syntax tree (AST),
        and then walks through the AST. It identifies and blanks out all docstrings and
        multiline string literals. Additionally, due to the nature of AST parsing in Python,
        comments (which start with '#') are not included in the AST and hence are not present
        in the unparsed code, effectively removing them from the output.
    
        It's important to note that this removal of comments is a byproduct of how the Python
        parser and the AST handle comments, rather than an explicit action by this function.
    
        Args:
            code (str): A string representing the Python code to be distilled.
    
        R

In [40]:
print(A_GIS.Code.highlight(code=content,force=True))

import A_GIS.Code.distill

def test_distill_removes_docstrings():
    test_code = '\n    def example_function(param1, param2):\n        """This is a docstring."""\n        return (param1, param2)\n    '
    expected_result = "\n    def example_function(param1, param2):\n        return (param1, param2)\n    "
    assert A_GIS.Code.distill.distill(code=test_code) == expected_result

def test_distill_removes_multiline_string_literals():
    ds = '"' * 3 + "I am a docstring!" + '"' * 3
    test_code = f"\n    def example_function(param1, param2):\n        {ds}\n        return (param1, param2)\n    "
    expected_result = "\n    def example_function(param1, param2):\n        return (param1, param2)\n    "
    assert A_GIS.Code.distill.distill(code=test_code) == expected_result

def test_distill_removes_comments():
    test_code = "\n    def example_function(param1, param2):\n        # This is a comment.\n        return (param1, param2)\n    "
    expected_result = "\n    def example_functio